In [ ]:
import re, os, shutil
from bs4 import BeautifulSoup
from glob import glob

def IO():
    # input
    outputpath = 'output'

    print('Remove \'output\' dir if exist...')
    if os.path.isdir("output"):
        shutil.rmtree(f'./{outputpath}')
    
    print('mkdir \'output\'')
    os.mkdir(outputpath)
    os.mkdir(f'{outputpath}/images')

    files = glob('./*.html')
    files = [os.path.basename(f'{file}').split('.')[0] for file in files]

    for filename in files:
        file =  open(f'./{filename}.html', 'r', encoding='utf8')
        contents = file.read()
        
        print(f'Read file: {filename}')

        soup = BeautifulSoup(contents, features='html.parser')

        # output
        outline, abs, kwd, img, tbl, body = context(soup, filename)

        output = outline + "</div><hr>"
        output += "<div id='articleBody'>"
        output += '''<div id="articleBody">
                    <h3 id="Abstract" class="sectionTitle">Abstract</h3>''' + abs
        output += kwd
        output += body

        # Write file
        write_html = f'./{outputpath}/{filename}.html'
        print(f'Write html: {write_html}')

        f = open(write_html, 'w', encoding='utf8')
        f.write(output)
        f.close()

        if img:
            write_img = f'./{outputpath}/{filename}_img.html'
            print(f'Write img html: {write_img}')
            f = open(write_img, 'w', encoding='utf8')
            f.write(img)
            f.close()
        
        if tbl:
            write_tbl = f'./{outputpath}/{filename}_tbl.html'
            print(f'Write img table: {write_tbl}')
            f = open(write_tbl, 'w', encoding='utf8')
            f.write(tbl)
            f.close()
    
    print('Completed!')


def counter(count):
    return count+1

def abstracts(p):
    paragraph = str(p)
    paragraph = sup_and_sub(paragraph)

    regex = r'^<p class=\".*摘文.*?\">(.*)<\/p>'
    subst = r'\1'
    paragraph = re.sub(regex, subst, paragraph)


    #有標題的摘要
    if(re.search(r'^<span class=\"TNR-B\">', paragraph)):
        regex = r'^<span class=\"TNR-B\">(.*?):<\/span> (.*)'
        paragraph = re.findall(regex, paragraph)
        paragraph = f'<p>{paragraph[0][0]}</p><p>{paragraph[0][1]}</p>'
        return paragraph
    #無標題的摘要
    else:
        paragraph = f'<p>{paragraph}</p>'
        return paragraph


def sup_and_sub(paragraph):
    # 上標
    ## 引文
    regex = r'<span class=\"上標.*?\">(.*?)<\/span>'
    subst = "<a href=\"#ref\\1\" id=\"\\1\" name=\"\\1\"><sup>\\1</sup></a>"
    paragraph = re.sub(regex, subst, paragraph)
    
    ## 處理多個引用的超連結"," or"-"
    regex = r'\"#ref(\d+)(\W)\d+.*?\"'
    subst = r'"#ref\1"'
    paragraph = re.sub(regex, subst, paragraph)

    ## 刪除非數字的上標超連結
    regex = r'<a href=\"#ref(\D+?)\".*?(<sup>\D+?</sup>)</a>'
    subst = r'\2'
    paragraph = re.sub(regex, subst, paragraph)
    
    # 下標
    regex = r'<span class=\"下標.*?\">(.*?)<\/span>'
    subst = r'<sub>\1</sub>'
    paragraph = re.sub(regex, subst, paragraph)

    # 斜體
    regex = r'<span class="TNR-I">(.*?)<\/span>'
    subst = r'<l>\1</l>'
    paragraph = re.sub(regex, subst, paragraph)

    return paragraph

# 內文
def context(soup, filename):
    contextOutput = """<!-- 內文 -->
                        <hr>
                       <!--hr-->"""
    context = soup.find_all('p')

    outlineString = '''<div class="outline">
                       <h3 id="Outline" class="selectionTitle">Outline</h3>'''
    
    absString = ''
    kwdString = ''
    imgString = ''
    tblString = ''
    temp = []
    titleCount = 0
    refCount = 0

    for p in context:

        if '摘文' in p.attrs['class'][0]:
            outlineString = '<div class="TitleL">．<a href="#Abstracts">Abstracts</a></div>'
            absString += abstracts(p)
            
        if '關鍵字' in p.attrs['class'][0]:
            outlineString += '<div class="TitleL">．<a href="#Keywords">Keywords</a></div>'
            regex = 'Keywords: '
            kwdString = re.sub(regex, '',p.text)
            kwdString = f'''<!-- Keywords -->
                            <h3 id='Keywords' class='sectionTitle'>Keywords</h3>
                            <p>{kwdString}</p>'''

        if '-壹標-' in p.attrs['class'][0]:
            
            if p.text == 'References':
                formatString = '''<!--/#articleBody-->
                                    <!-- References -->
                                    <hr>
                                    <h3 id="References" class="sectionTitle">References</h3>
                                    <div class="referencesWrap">
                                    <table cellpadding="0" cellspacing="0" border="0">
                                    <tbody>'''
                outlineString += '<div class="TitleL">．<a href="#References">References</a></div>'
            
            else:
                titleCount = counter(titleCount)
                id = f"paragraph{titleCount}"
                formatString = f'<p id="{id}" style="font-size:22px;color:#000;"><b>{p.text}</b></p>'
                outlineString += f'<div class="TitleL">．<a href="#{id}">{p.text}</a></div>'

            temp.append(formatString)

        if '-一標-' in p.attrs['class'][0]:
            formatString = f'<p><b>{p.text}</b></p>'
            temp.append(formatString)

        if '內文' in p.attrs['class'][0]:
            paragraph = str(p)
            paragraph = sup_and_sub(paragraph)
            
            regex = r'<p class=\"內文.*?\">'
            subst = "<p>"
            paragraph = re.sub(regex, subst, paragraph)

            temp.append(paragraph)

        if('圖標')  in p.attrs['class'][0]:
            
            paragraph = str(p)
            paragraph = sup_and_sub(paragraph)

            regex = r'Figure (?P<id>\d+)'
            m = re.search(regex, paragraph)
            id = m.group('id')

            regex = r'</span>(?P<title>.*?)</p>'
            m = re.search(regex, paragraph)
            title = m.group('title')

            formatString = f'''<div class="imgItemWrap">
                                <div class="imgItem" align="center" id="fig{id}">
                                    <a class="btnImgFullSize" href="images/{filename}_fig_{id}.jpg" target="_blank">
                                        <img src="images/{filename}_fig_{id}.jpg" alt="Figure {id}." style="width: 400px;"><br>
                                        <span>Download full-size image</span>
                                    </a>
                                <div class="imgDesc imgDescDown">Fig. {id}. {title}'''

            if p.find_next('p'):
                nextP = p.find_next('p')

                if ('表註') in nextP.attrs['class'][0]:
                    paragraph = str(nextP)
                    paragraph = sup_and_sub(paragraph)

                    regex = r'>(?P<note>.*)</p>'
                    m = re.search(regex, paragraph)
                    note = m.group('note')

                    formatString += f'<br>{note}'
            
            formatString += '</div></div></div>'

            imgString += formatString

        if('表標')  in p.attrs['class'][0]:

            paragraph = str(p)
            paragraph = sup_and_sub(paragraph)

            regex = r'Table (?P<id>\d+)'
            m = re.search(regex, paragraph)
            id = m.group('id')

            regex = r'</span>(?P<title>.*?)</p>'
            m = re.search(regex, paragraph)
            title = m.group('title')

            formatString =  f'''<div class="imgItemWrap">
                                <div class="imgItem" align="center">
                                    <div id="table{id}" class="imgDesc imgDescUp">Table {id}. {title}</div>
                                        <a class="btnImgFullSize" href="images/{filename}_tbl_{id}.jpg" target="_blank">
                                    <img src="images/{filename}_tbl_{id}.jpg" alt="Table {id}." style="width: 900px;"><br>
                                    <span>Download full-size image</span>
                                </a>
                              </div>
                            </div>'''
            
            tblString += formatString


    # Reference
    context = soup.find_all('li')

    for li in context:
        if '-參文-' in li.attrs['class'][0]:
            paragraph = str(li)
            paragraph = sup_and_sub(paragraph)

            
            regex = r'<li class=\"_-參文-英-\".*?\"?>(?P<authors>.*?\.) (?P<title>.*?[\.|\?])'
            m = re.search(regex, paragraph)
            if m:
                authors, title = m.group('authors'), m.group('title')
            else:
                authors, title = 'None', 'None'
            
            regex = r'<span class=\"參文-英\">(?P<journal>.*)</span>. (?P<info>\d{4};.*?\.)'
            m = re.search(regex, paragraph)
            if m:
                journal, info = m.group('journal'), m.group('info')
            else:
                journal, info = 'None', 'None'

            regex = r'doi:(?P<doi>.*)</li>'
            m = re.search(regex, paragraph)
            if m:
                doi = m.group('doi')
            else:
                doi = 'None'

            regex = r'<span class=\"超連結\">(?P<url>.*)</span></a>.*? (?P<urlinfo>.*)</li>'
            m = re.search(regex, paragraph)
            if m:
                url, urlinfo = m.group('url'), m.group('urlinfo')
            else:
                url, urlinfo = 'None', 'None'
                
                        

            refCount = counter(refCount)

            formatString = f'''<tr>
                                <td valign="top">
                                    <strong id="ref{refCount}" class="rNumber"><a href='#{refCount}'>{refCount}</a></strong>
                                </td>
                                <td valign="top">
                                    <div class="refContent">
                                    <div class="rAuthors">{authors}</div>
                                    <div class="rTitle"><b>{title}</b></div>'''
            if(journal != 'None'):
                formatString += f'<div class="rInformation">{journal}. {info}</div>'

            elif(url != 'None'):
                formatString += f'<div class="rInformation"><a href="{url}">{url}.</a>. {urlinfo}</div>'

            if(doi != 'None'):
                formatString += f'<div class="linkAndFile"><a href="https://doi.org/{doi}" target="_blank">CrossRef</a></div>'

            formatString += f'''</div></td></tr>'''

            temp.append(formatString)
    
    temp.append('</tbody></table></div>')




    
    contextOutput += ' '.join(temp)

        
    return outlineString, absString, kwdString, imgString, tblString, contextOutput



IO()